In [1]:
import pandas as pd
import mysql.connector 
from sqlalchemy import create_engine 

In [3]:
#1
cnx = mysql.connector.connect(user="root",
                             password = "lobeckkreuzberg",
                             host = "localhost",
                             database = "bank")

In [5]:
cnx.is_connected()

True

In [7]:
#2
def rentals_month(engine, month, year):
    """
    Retrieves rental data for the specified month and year from the Sakila database.
    """
    # SQL query to retrieve rental data for the given month and year
    query = f"""
    SELECT * 
    FROM rental 
    WHERE EXTRACT(MONTH FROM rental_date) = {month}
    AND EXTRACT(YEAR FROM rental_date) = {year};
    """
    # Execute the query and return the results as a DataFrame
    return pd.read_sql(query, engine)

In [9]:
#3
def rental_count_month(df, month, year):
    """
    Takes a DataFrame with rental data and returns a new DataFrame containing
    the number of rentals made by each customer_id during the specified month and year.
    """
    # Create a new column name based on the month and year
    column_name = f"rentals_{str(month).zfill(2)}_{year}"
    
    # Group the DataFrame by customer_id and count the number of rentals
    rental_counts = df.groupby('customer_id')['rental_id'].count().reset_index()
    
    # Rename the column to reflect the month and year
    rental_counts.rename(columns={'rental_id': column_name}, inplace=True)
    
    return rental_counts

In [11]:
#4
def compare_rentals(df1, df2):
    """
    Compares the number of rentals made by each customer in two different months and years.
    """
    
    # Merge the two DataFrames on customer_id
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2'))
    
    # Fill missing values with 0 (if a customer rented in one month but not the other)
    combined_df.fillna(0, inplace=True)
    
    # Create a new 'difference' column showing the difference in rental counts
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    
    return combined_df